In [1]:
import pandas as pd

def load_public_sheet(sheet_url):
    # Convert Google Sheets URL to CSV export URL
    sheet_id = sheet_url.split('/d/')[1].split('/')[0]
    csv_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv"
    
    # Read directly with pandas
    df = pd.read_csv(csv_url)
    return df

In [2]:
data = load_public_sheet("https://docs.google.com/spreadsheets/d/1W4EtybMasTRdjprITFIDiGv4sNLzIGR8ADiUWQFfhy8/edit?usp=sharing")
print (data.head())

   STT                                         Tên Đề Tài  \
0    1  Development of a wireless sensor network for a...   
1    2  Development of a NEMA Controller for Smart Str...   
2    3  Phát triển ứng dụng VR cho việc giám sát và đi...   
3    4  Tích hợp công nghệ NBIoT cho đồng hồ nước điện tử   
4    5  Xây dựng hệ thống hỗ trợ đăng kí và check in c...   

                        Link Canva Report (Editable)  \
0  https://www.canva.com/design/DAGF6z2n4H8/0GIMc...   
1  https://www.canva.com/design/DAGFfInv7R0/oCL61...   
2                           Thesis VR in Agriculture   
3  https://www.canva.com/design/DAFvmua0V-8/jyQEf...   
4  https://www.canva.com/design/DAGCWSa4mB0/oy1oc...   

                                         Link Github  \
0  https://github.com/ACLAB-HCMUT/MDT-236-Capston...   
1                                                NaN   
2            Final source for VR Agriculture project   
3            https://github.com/phucho123/DATN_Final   
4    https://git

In [3]:
# Test Google API credentials
from google.oauth2 import service_account
from googleapiclient.discovery import build

# Path to your credential file
SERVICE_ACCOUNT_FILE = "automation-workflow-project-73a3cbdd2942.json"
# Define the scopes required
SCOPES = ['https://www.googleapis.com/auth/drive.metadata.readonly']

# Authenticate and build the service
creds = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES)

service = build('drive', 'v3', credentials=creds)

# List the first 10 files in your Google Drive
results = service.files().list(pageSize=10, fields="files(id, name)").execute()
items = results.get('files', [])

if not items:
    print('No files found.')
else:
    print('Files:')
    for item in items:
        print(f"{item['name']} ({item['id']})")

Files:
Signed Forms (1oAngjLvLnwYkL3dgZ0cBsyjtulr0GVjI)


In [ ]:
import gspread
from google.oauth2.service_account import Credentials

SERVICE_ACCOUNT_FILE = "automation-workflow-project-73a3cbdd2942.json"
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']

creds = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
gc = gspread.authorize(creds)

# Replace with your Google Sheet ID
SHEET_ID = "1W4EtybMasTRdjprITFIDiGv4sNLzIGR8ADiUWQFfhy8"
sh = gc.open_by_key(SHEET_ID)
worksheet = sh.sheet1
print("First row of sheet:", worksheet.row_values(1))

FileNotFoundError: [Errno 2] No such file or directory: 'automation-workflow-project-73a3cbdd2942.json'

In [ ]:
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build

SERVICE_ACCOUNT_FILE = "automation-workflow-project-73a3cbdd2942.json"
SCOPES = ['https://www.googleapis.com/auth/drive.metadata.readonly']

creds = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
service = build('drive', 'v3', credentials=creds)

results = service.files().list(pageSize=5, fields="files(id, name)").execute()
items = results.get('files', [])
print("Drive files:", items)

Drive files: []


In [ ]:
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
import os

SERVICE_ACCOUNT_FILE = "automation-workflow-project-73a3cbdd2942.json"
SCOPES = ['https://www.googleapis.com/auth/drive.file']

creds = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
service = build('drive', 'v3', credentials=creds)

folder_id = '1oAngjLvLnwYkL3dgZ0cBsyjtulr0GVjI'
file_path = 'test_upload.txt'

# Create a test file to upload
with open(file_path, 'w') as f:
    f.write('Hello from Google Drive API!')

file_metadata = {
    'name': os.path.basename(file_path),
    'parents': [folder_id]
}
media = MediaFileUpload(file_path, resumable=True)

uploaded_file = service.files().create(
    body=file_metadata,
    media_body=media,
    fields='id, name'
).execute()

print(f"Uploaded file: {uploaded_file['name']} (ID: {uploaded_file['id']})")

HttpError: <HttpError 403 when requesting https://www.googleapis.com/upload/drive/v3/files?fields=id%2C+name&alt=json&uploadType=resumable returned "Service Accounts do not have storage quota. Leverage shared drives (https://developers.google.com/workspace/drive/api/guides/about-shareddrives), or use OAuth delegation (http://support.google.com/a/answer/7281227) instead.". Details: "[{'message': 'Service Accounts do not have storage quota. Leverage shared drives (https://developers.google.com/workspace/drive/api/guides/about-shareddrives), or use OAuth delegation (http://support.google.com/a/answer/7281227) instead.', 'domain': 'usageLimits', 'reason': 'storageQuotaExceeded'}]">

In [1]:
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
import os

SCOPES = ['https://www.googleapis.com/auth/drive.file']
CLIENT_SECRET_FILE = 'client_secret_608833324413-fkq10e04u2pht5ksua1rm7j6bjtpeduo.apps.googleusercontent.com.json'

# Run local server for authentication
flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRET_FILE, SCOPES)
creds = flow.run_local_server(port=8080)

service = build('drive', 'v3', credentials=creds)

folder_id = '1oAngjLvLnwYkL3dgZ0cBsyjtulr0GVjI'  # Your folder ID
file_path = 'test_upload.txt'

# Create a test file to upload
with open(file_path, 'w') as f:
    f.write('Hello from Google Drive API with OAuth!')

file_metadata = {
    'name': os.path.basename(file_path),
    'parents': [folder_id]
}
media = MediaFileUpload(file_path, resumable=True)

uploaded_file = service.files().create(
    body=file_metadata,
    media_body=media,
    fields='id, name'
).execute()

print(f"Uploaded file: {uploaded_file['name']} (ID: {uploaded_file['id']})")

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=608833324413-fkq10e04u2pht5ksua1rm7j6bjtpeduo.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.file&state=PTireELNnJerYYOfpVuib4NSQzjpO8&access_type=offline
Uploaded file: test_upload.txt (ID: 1p3hbgiFNSTOF2Tw9F0Wa0DBk1qkaCI_n)


In [1]:
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from email.mime.text import MIMEText
import base64

SCOPES = ['https://www.googleapis.com/auth/gmail.send']
CLIENT_SECRET_FILE = 'client_secret_608833324413-fkq10e04u2pht5ksua1rm7j6bjtpeduo.apps.googleusercontent.com.json'

# Authenticate and build the Gmail service
flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRET_FILE, SCOPES)
creds = flow.run_local_server(port=8080)
service = build('gmail', 'v1', credentials=creds)

# Prepare the email
to = "hieu.nguyenminh03@hcmut.edu.vn"  # Replace with recipient's email
subject = "Test Email from Gmail API"
body = "Hello, this is a test email sent from Python using Gmail API!"

message = MIMEText(body)
message['to'] = to
message['subject'] = subject

# Encode the message
raw_message = base64.urlsafe_b64encode(message.as_bytes()).decode()

# Send the email
send_message = service.users().messages().send(
    userId="me",
    body={'raw': raw_message}
).execute()

print(f"Email sent! Message ID: {send_message['id']}")

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=608833324413-fkq10e04u2pht5ksua1rm7j6bjtpeduo.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.send&state=erHA8GrE5LNSIARynMtgRvi4IUDmDr&access_type=offline
Email sent! Message ID: 198851ecf756f052
